In [9]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
import matplotlib as mpl
import matplotlib.pyplot as plt
import requests # library to handle requests
import random # library for random number generation

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
import folium

#! conda install -c conda-forge ipywidgets --yes
import ipywidgets as widgets

print('Folium installed and imported!')

Folium installed and imported!


In [10]:
# @hidden_cell
CLIENT_ID = ' ' # your Foursquare ID
CLIENT_SECRET = ' ' # your Foursquare Secret
VERSION = '20200501'
LIMIT = 100 # max only 50 result 

radius = 10000

In [11]:
delhi_district_file_path='./delhi_all.csv'
delhi_district=pd.read_csv(delhi_district_file_path)

delhi_population_path='./delhi9_population.csv'
delhi_population=pd.read_csv(delhi_population_path)

delhi_geo_path='./Delhi.geojson'
import json
#delhi_geo = json.loads(delhi_geo_path)
with open(delhi_geo_path, 'r') as j:
     delhi_geo = json.loads(j.read())
        
#delhi_geo=pd.read_csv(delhi_geo_path, error_bad_lines=False)

In [12]:
 delhi_district.place_name.unique()
# delhi_population.head(3)
# delhi_geo.columns
# print(delhi_geo)

array(['Connaught Place', 'Darya Ganj', 'Aliganj', 'Rashtrapati Bhawan',
       'Lower Camp Anand Parbat', 'Bara Tooti', 'Birla Lines',
       'Patel Nagar', 'Delhi Cantt', 'Nirman Bhawan', 'Inderpuri',
       'Hazrat Nizamuddin', 'Jangpura', 'Zakhira', 'Hauz Khas',
       'Malviya Nagar', 'Vishnu Garden', 'Nehru Place',
       'Flatted Factories Complex', 'Malcha Marg', 'Postal Saving Bureau',
       'Kidwai Nagar', 'Lajpat Nagar', 'Jamia Nagar', 'Punjabi Bagh',
       'J 6block Rajouri Garden', 'Naraina Industrial Estate',
       'Himayunpur Extn', 'T B Hospital', 'Gandhi Nagar', 'Shahdara',
       'Adarsh Nagar', 'Pitampura', 'Inderlok', 'Alipur', 'Gurgaon Road',
       'A F Rajokari', 'Bawana', 'Sanoth', 'Nagloi', 'Badli', 'Najafgarh',
       'Badarpur T P Station', 'Palam Enclave', 'Nangal Rava',
       'Arjan Garh', 'Kailash', 'Andrews Ganj', 'Azad Nagar',
       'Wazirpur Phase Iii', 'Zafrabad', 'Civil Lines', 'Paharganj',
       'Shakurbasti', 'Munirka', 'Janakpuri', 'Uttam Nag

In [13]:
from IPython.display import clear_output
caption=widgets.Label(value='Not selected yet')
ALL='ALL'
def unique_sorted_values_ALL(array):
    unique=array.unique().tolist()
    unique.sort()
    unique.insert(0,ALL)
    return unique
x='JAMES'
def onchange(d):
    #clear_output(wait=False)
    #print ('%s' % d['new'] )
    pass
    #print('Selection changed event fired.')

district=widgets.Dropdown(
    options=delhi_district.place_name,
    description='Places :'
)
widgets.link((district,'value'),(caption,'value')) 
district.observe(onchange)
display(district, caption) 

Dropdown(description='Places :', options=('Connaught Place', 'Darya Ganj', 'Aliganj', 'Rashtrapati Bhawan', 'L…

Label(value='Connaught Place')

In [14]:
vars(district); 
state=district.value
selectedRow=delhi_district.loc[delhi_district['place_name']==district.value, :]

vars(selectedRow);

In [15]:
geolocator = Nominatim(user_agent="foursquare_agent", format_string="%s, New delhi,IN")
# location = geolocator.geocode(district.value)
# location = geolocator.reverse("28.509669, 77.376294")
# latitude = location.latitude
# longitude = location.longitude

latitude = selectedRow.loc[district.index,'latitude']
longitude =selectedRow.loc[district.index,'longitude']
location = geolocator.reverse("{}, {}".format(latitude, longitude))
 
# location.raw

In [16]:
# Delhi latitude and longitude values
delhi_map=folium.Map(location=[latitude, longitude], zoom_start=10.47)

# Add the color for the chloropleth:
delhi_map.choropleth(
 geo_data=delhi_geo,
 name='choropleth',
 data=delhi_population,
 columns=['District', 'Percentage'],
 key_on='feature.properties.Dist_Name',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Population (%)'
)
# folium.LayerControl().add_to(delhi_map)


# I can add marker one by one on the map
for i in range(0,len(delhi_district)):
    folium.Marker([delhi_district.iloc[i]['latitude'], delhi_district.iloc[i]['longitude']], popup=folium.Popup(delhi_district.iloc[i]['place_name'], parse_html=True, max_width=100) ).add_to(delhi_map)

delhi_map

C:\ProgramData\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [17]:
search_query = 'market'
url = 'https://api.foursquare.com/v2/search/recommendations?client_id={}&client_secret={}&ll={},{}&v={}&query={}&limit={}&state={}&radius={}&sortByDistance=True'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, LIMIT, state, radius)
url
# search/recommendations

'https://api.foursquare.com/v2/search/recommendations?client_id=OLGSFJEPQVYF5JHRQ0SEATOXPYZXURLTVXK4222KZQYXATRY&client_secret=NI0RLJZPQ0POAJBHCTXKXPECSRMPBZU5SAPMIV1SIF2N30SU&ll=28.6333,77.2167&v=20200501&query=market&limit=100&state=Connaught Place&radius=10000&sortByDistance=True'

In [18]:
results = requests.get(url).json()
results;

In [19]:
# assign relevant part of JSON to venues
venues = results['response']['group']['results']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe=dataframe[dataframe['venue.location.distance']<=5000]

dataframe.head()
# dataframe['location.address']


,displayType,id,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,...,photo.createdAt,photo.prefix,photo.suffix,photo.width,photo.height,photo.visibility,snippets.count,snippets.items,venue.location.crossStreet,venue.location.neighborhood
0,venue,5eadcc906d5b5400081b799f,4bb4c55d6ebfc9b6db56edfa,Palika Bazaar (Palika Market),Connaught Place,28.631253,77.219052,"[{'label': 'display', 'lat': 28.63125308558757...",323,India,...,1.380357e+09,https://fastly.4sqi.net/img/general/,/138616_3Zug8X0NOj2q8efjEo5CxmdG4dYaUm4LdIo2Bk...,960.0,925.0,public,1,"[{'detail': {'type': 'tip', 'object': {'id': '...",NaN,NaN
1,venue,5eadcc906d5b5400081b79a0,4d1879c8e190721eee1f2221,Janpath Street Market,Janpath,28.628645,77.219159,"[{'label': 'display', 'lat': 28.62864495388877...",571,110001,...,1.368694e+09,https://fastly.4sqi.net/img/general/,/5417987_c53UWE3Xy8H7vF_CbSd3jlof4IgybhWjI9w3T...,720.0,960.0,public,1,"[{'detail': {'type': 'tip', 'object': {'id': '...",Nr. Connaught Place,NaN
2,venue,5eadcc906d5b5400081b79a1,4d71fd224ab5224b5a49b897,"Shankar Market, CP",Connaught Place,28.634071,77.223888,"[{'label': 'display', 'lat': 28.63407138821373...",707,NaN,...,1.517048e+09,https://fastly.4sqi.net/img/general/,/50171199_Ukp5NBytEYYEj05onaBSiMGzghxen80J0OwJ...,1440.0,1920.0,public,1,"[{'detail': {'type': 'tip', 'object': {'id': '...",NaN,NaN
3,venue,5eadcc906d5b5400081b79a2,4e0f245152b1b27c1b8d8268,Panchkuian Furniture Market,"Bhai Vir Singh Marg, Sector 4, Gole Market",28.638270,77.209828,"[{'label': 'display', 'lat': 28.63826999000177...",869,110001,...,1.452174e+09,https://fastly.4sqi.net/img/general/,/34556529_P6HxNwuoxAyUMYj5Wizp8VbBN0y2rVw5qREe...,720.0,960.0,public,1,[{}],Near Petrol Station,NaN
4,venue,5eadcc906d5b5400081b79a3,51074fcce412de41b62c2cb1,Tibetan Market,NaN,28.625740,77.219059,"[{'label': 'display', 'lat': 28.62574046725315...",872,NaN,...,1.512627e+09,https://fastly.4sqi.net/img/general/,/37873733_x2keWvxsvqDCKXsGGm8gMJG2jeRzqmB6WsWZ...,1900.0,1425.0,public,1,[{}],Janpath,NaN
